In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import sys
import warnings
warnings.filterwarnings("ignore")

In [4]:
#options = EdgeOptions()
driver = webdriver.Edge()
driver.maximize_window()
data = pd.DataFrame()

driver.get("https://www.forbes.com/money/")
close = driver.find_element(By.XPATH, value="/html/body/div[8]/div/button")
close.click()
delay = 10 # seconds

count = 1   
#change 200 to other number if needs, as that is indicator for extract more news or less news depend on how many header an desc you look for.

for a in range(1,199):
    # Output to terminal repetitively over the same line using '\r'.
    sys.stdout.write("\rClick More Button Progress: {0}".format(a))
    sys.stdout.flush()
    more_articles = driver.find_element(By.XPATH, value="//button[contains(text(),'More Articles')]")
    more_articles.click()
    wait = WebDriverWait(driver, 10)
    more = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'More Articles')]")))
    driver.implicitly_wait(0.5)
    
for i in range(1,2000):
    sys.stdout.write("\rItem: {0}".format(count))
    sys.stdout.flush()
    try:
        #print('get value {0}'.format(count))
        header = driver.find_element(By.XPATH, value="//*[@id=\"row-2\"]/div/div/div/div[1]/div/div/article[{0}]/div[1]/h3/a".format(count))
        desc = driver.find_element(By.XPATH, value="//*[@id=\"row-2\"]/div/div/div/div[1]/div/div/article[{0}]/div[1]/div[2]".format(count)) 
        author_label = driver.find_element(By.XPATH, value="//*[@id=\"row-2\"]/div/div/div/div[1]/div/div/article[{0}]/div[1]/div[3]/div/span/div/div/a".format(count))
        author = author_label.text
    except:
        author = ""
        header = ""
        desc = ""
            #print('Error 1')

    date_value = None
    try:
        date_element = driver.find_element(By.XPATH, value="/html[1]/body[1]/div[1]/main[1]/div[1]/section[1]/section[4]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/article[{0}]/div[1]/div[1]".format(count))

        if date_element.is_displayed():
            date_value = date_element.text
        else:
            date_element = driver.find_element(By.XPATH, value="/html[1]/body[1]/div[1]/main[1]/div[1]/section[1]/section[4]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/article[{0}]/div[1]/div[2]".format(count))
            date_value = date_element.text


    except:
        date_value=""
        #print('Error 2')

    if header != "":
        #print('store value {0}'.format(count))
        record = pd.DataFrame({
                                    'header':header.text,
                                    'desc': desc.text,
                                    'author': author,
                                    'date': date_value
                                }, index=[0])
        data = pd.concat([data, record],ignore_index=True, axis=0)
    count = count + 1
    
print('Completed load all articles.')

#driver.quit()
data.reset_index()
#data.drop(columns=data.columns[0], axis=1, inplace=True)
data.to_csv(r"Data/Forbes/{0}_new_data.csv".format('forbes_money_news_raw'), sep="\t")
data

Item: 1999 Button Progress: 198Completed load all articles.


,header,desc,author,date
0,How To Keep Your Self-Employment Income Rising...,"With inflation rising, the self-employed must ...",Ryan Derousseau,5 minutes ago
1,A Brutal Quarter,"In pre-market trading, instead of a day of win...",JJ Kinahan,16 minutes ago
2,Forget Gas Tax Holidays. Bigger State Tax Cuts...,With the fiscal year starting July 1 and a bud...,Gabriela Lopez Gomes,22 minutes ago
3,How The Price Is Right Predicted This Bear Market,Just when I thought 2022 couldn’t get any stra...,Brett Owens,26 minutes ago
4,"Bearish Sentiment Improves, But Remains Elevated",Investors are bearish by a factor of two-to-one.,Simon Constable,28 minutes ago
...,...,...,...,...
1776,Has Inflation Peaked? Fed’s Favorite Indicator...,One closely watched inflation indicator just f...,Jonathan Ponciano,"Apr 29, 2022"
1777,Elon Musk Sells More Tesla Stock—Bringing Tota...,Musk offloaded another 5.2 million Tesla share...,Sergei Klebnikov,"Apr 29, 2022"
1778,American International Group Stock To Pass The...,The premiums grew 189% y-o-y to $2.7 billion i...,Trefis Team,"Apr 29, 2022"
1779,First Quarter GDP Was Stronger Than The Report...,While the first quarter saw a reported decline...,Chuck Jones,"Apr 29, 2022"


# Data Processing
Manual process to clean up the data that extract from above script. We replace the data to a proper format such as 2 hours ago to specific date that extract on that day in order the data can be use in time series analysis later. 

In [5]:
import datetime as dt
from datetime import date
import numpy as np

news = pd.read_excel(r"Data/Forbes/Forbes_News_Jun2022.xlsx")

In [6]:
news

,Column1,header,desc,author,date
0,0,"Meet The Vermont Startup That Amazon, Gucci, A...","Superplastic, founded by Paul Budnitz in 2017,...",Marty Swant,2022-04-29 00:00:00
1,1,"Stock Market Fear Is Overdue, So Be Ready For ...",Wall Street is waiting for investor fear to sh...,John S. Tobey,2022-04-29 00:00:00
2,2,Getting A Handle On Inflation Affects How Far ...,"For Gen Xers, inflation can impact their retir...",Next Avenue,2022-04-29 00:00:00
3,3,After Widowhood Comes A Second Act With Purpose,What I learned after my husband’s death helped...,Next Avenue,2022-04-29 00:00:00
4,4,A Hopeful Future For Midlife Entrepreneurship,Read the inspiring stories of three mid-life e...,Next Avenue,2022-04-29 00:00:00
...,...,...,...,...,...
9117,9117,Working From Home And Worker Bargaining Power,Does teleworking signal a power shift between ...,Richard McGahey,"Apr 30, 2022"
9118,9118,7 Ways To Save Extra Cash Without Sacrificing ...,If your budget is feeling the strain of inflat...,Eric Roberge,"Apr 30, 2022"
9119,9119,This Surging New Cryptocurrency Could Rocket H...,A survey of cryptocurrency experts has returne...,Billy Bambrough,"Apr 30, 2022"
9120,9120,"New Ledgers, New Business Models And New Oppor...",The examples of both Dropp and Helium seem to ...,David G.W. Birch,"Apr 30, 2022"


### Manual process to label news category.
Split the news into crypto, finance, stock market and metaverse category. Break down to specific news in order for us to find what news is more sensitive for price impact. 

In [10]:
start = dt.datetime(2021,10,1)
end = dt.datetime(2022,6,30)

def filter_news_by_date(start, end, data):
    data.date = pd.to_datetime(data.date)
    data = data[(data['date'] >= start) & (data['date'] <= end)]
    data = data.set_index('date') 
    data = data.sort_index()
    data = data.reset_index()
    #data = data.set_index('date')
    return data

In [21]:
news = pd.read_excel(r"Data/Forbes/Forbes_News_Jun2022.xlsx")
news = news.drop(['Column1'], axis=1)
news['category'] = np.where(news.category == 'Metaverse', 'Cryptocurrencies', news.category)
news_target_date = filter_news_by_date(start, end, news)
news_target_date

,date,category,header,desc,author
0,2021-10-06,Finance,India’s Wealth Creation: Rapid Recovery Puts E...,GDP grew at a record pace in the fiscal first ...,Rainer Michael Preiss
1,2021-10-06,Stock Market,McConnell Offers Democrats An Emergency Debt L...,"For weeks, the top Senate Republican has firml...",Jonathan Ponciano
2,2021-10-06,Finance,How Making Public Long-Term Care Insurance (So...,When Washington State legislators approved a p...,Howard Gleckman
3,2021-10-06,Stock Market,Blackstone Mortgage Trust Passes Through 8% Yi...,"In trading on Wednesday, shares of Blackstone ...",Dividend Channel
4,2021-10-06,Finance,Why Traders Should Dig For Goldman Sachs Stock...,The shares of Goldman Sachs (GS) are up 0.4% a...,Schaeffer's Investment Research
...,...,...,...,...,...
9117,2022-06-30,Stock Market,3 Key Student Loan Forgiveness Opportunities M...,Several new student loan forgiveness initiativ...,Adam S. Minsky
9118,2022-06-30,Cryptocurrencies,Supplementary Review: Progress Software (PRGS)...,Fiscal 2022 Q2 adjusted revenue (includes acqu...,Taesik Yoon
9119,2022-06-30,Stock Market,"Portfolio Change: One Stock To Buy, June 29",Investors’ fears that consumer demand will slo...,Taesik Yoon
9120,2022-06-30,Finance,"Private Equity, Crypto Allowed In 401ks—Lotter...",Now that 401(k) investors are now free to jeop...,Edward Siedle


In [23]:
news_target_date.groupby(['category']).size().reset_index(name='counts')

,category,counts
0,Cryptocurrencies,1129
1,Finance,5596
2,Stock Market,2397
